In [0]:
# Guardar y cargar modelos con TF

!pip install -q pyyaml h5py       # Libreria para persistir los datos en el HD
                                  # Esta especializada en manipular datos de Numpy
                                  # y tratar grandes volumenes de datos (Muy util para este tipo de redes)

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals  # Librerias para Colab (Solo para entornos de trabajo Colab)

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

1.15.0


In [4]:
# Para este ejemplo usaremos 1000 imagenes

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

# Definimos el modelo en un metodo

def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),                                        # Esta capa hace que en el forward alguna neuronas de la capa superior esten desactivadas, para reducir el Overfiting
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

# Creamos una instancia del modelo
model = create_model()

# Mostramos el modelo descrito
model.summary()


11493376/11490434 [==============================] - 0s 0us/step
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Ahora realizaremos la posibidad de hacer ceckpoint en nuestro modelo

# Creamos la variable para almacenar la ruta
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,  # Indicamos la direccion donde queremos almacenar los checkpoints
                                                 save_weights_only=True,    # Guarda solamenre cada epoca 
                                                 verbose=1)                 # Tipo de salida de informacion

model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])                                          # Pasamos por parametro el callback




Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 576/1000 [================>.............] - ETA: 1s - loss: 1.4886 - acc: 0.5816 
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 2s 2ms/sample - loss: 1.1593 - acc: 0.6760 - val_loss: 0.7018 - val_acc: 0.7920
Epoch 2/10
 576/1000 [================>.............] - ETA: 0s - loss: 0.4715 - acc: 0.8767
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 188us/sample - loss: 0.4248 - acc: 0.8810 - val_loss: 0.5213 - val_acc: 0.8370
Epoch 3/10
 640/1000 [==================>...........] - ETA: 0s - loss: 0.2984 - acc: 0.9250
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 147us/sample - loss: 0.2845 - acc: 0.9290 - val_loss: 0.4813 - val_acc: 0.8420
Epoch 4/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.2025 - acc: 0.9440
Epoch 00004: saving model to training_1/cp.ckpt
1000/100

In [12]:
# Comprobamos la salida
!ls {checkpoint_dir}

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


In [13]:
# Ahora vamos a crear otro modelo para cargar los datos preentrenados del modelo anterior
model = create_model()

# Evaluamos el modelo sin entrenar
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Su Accuracy es de:", acc*100, "%")

1000/1000 - 0s - loss: 2.3964 - acc: 0.0560
Su Accuracy es de: 5.60000017285347 %


In [14]:
# Cargamos los pesos del modelo preentrenado
model.load_weights(checkpoint_path)

# Volvemos a mirar su Accuracy
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Su Accuracy es de:", acc*100, "%")

1000/1000 - 0s - loss: 0.4392 - acc: 0.8620
Su Accuracy es de: 86.19999885559082 %


In [22]:
# Ahora podemos continuar entrenando a nuestra red nuevamente con el metodo fit :)

model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])   

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.0302 - acc: 1.0000
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 132us/sample - loss: 0.0295 - acc: 1.0000 - val_loss: 0.4148 - val_acc: 0.8750
Epoch 2/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.0241 - acc: 1.0000
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 156us/sample - loss: 0.0239 - acc: 0.9990 - val_loss: 0.4133 - val_acc: 0.8680
Epoch 3/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.0212 - acc: 1.0000
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 139us/sample - loss: 0.0210 - acc: 1.0000 - val_loss: 0.4146 - val_acc: 0.8780
Epoch 4/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.0160 - acc: 1.0000
Epoch 00004: saving model to training_1/cp.ckpt
1000/10

In [23]:
## Opciones interesantes de la clase Checkpoint

# Podemos inciar el tiop de epoca aqui abajo
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt" 
checkpoint_dir = os.path.dirname(checkpoint_path)

# Modificar parametros del metodo
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)                                         # Guardar cada 5 epocas

# Podemos guardar de forma manual los pesos mediante el metodo save_weights
model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))

# Realizamos las pruebas
model.fit(train_images, 
              train_labels,
              epochs=50, 
              callbacks=[cp_callback],
              validation_data=(test_images,test_labels),
              verbose=0)



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [28]:
# Podemos coger el ultimo weigt
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

# Nuevamente creamos el modelo y podemos pasarle los utimos pesos
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Su Accuracy es de:", acc*100, "%")

1000/1000 - 0s - loss: 0.6360 - acc: 0.8810
Su Accuracy es de: 88.09999823570251 %


In [32]:
# Para guardar los el modelo entero su arquitectura y pesos

model.save("modelo_guardado.h5")
!ls

# Cargamos el modelo en un modelo nuevo

nuevo_modelo = tf.keras.models.load_model('modelo_guardado.h5')
nuevo_modelo.summary()

modelo_guardado.h5  sample_data  training_1  training_2
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.
